# ทดสอบการค้นคืนข่าวที่น่าชื่อถือบน sure ก่อน share ด้วย cosine similarity + TF-IDF (ใช้เฉพาะหัวข้อข่าว(header จาก sure_info.json) 10/12/2021)

In [1]:
import numpy as np
import nltk
import re
import pandas as pd 
import json
import os
import pickle

from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

nltk.download('words')
th_stop = thai_stopwords()
en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if (not i in th_stop) & (not i in en_stop) & (not i in punctuation) & (not i in ["'",'"','“','”','‘','’','\n',"None", ' '])] 
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def read_sure_refer():
    # Opening JSON file
    root_path = os.getcwd()
    path = os.path.join(root_path, '../Scrapy/Scrapy_project/spiders//fetch file/sure_info.json')
    f = open(path, encoding="utf8")
    
    # returns JSON object as
    # a dictionary
    sure_refer_text_data = json.load(f)

    # Closing file
    f.close()
    return sure_refer_text_data

In [3]:
sure_refer_text_data = read_sure_refer()

In [4]:
len(sure_refer_text_data)

2068

In [5]:
def get_sure_refer_info(sure_refer_text_data): #สำหรับดึงหัวข้อข่าวอ้างอิง (header) และลิงของข่าวอ้างอิง (link)
    refer_text_list = []

    for refer_text in range(len(sure_refer_text_data)):
        tmp = []
        tmp.append(sure_refer_text_data[refer_text]['header'])
        tmp.append(sure_refer_text_data[refer_text]['link'])
        refer_text_list.append(tmp)

    return refer_text_list

In [6]:
sure_refer_text_list = get_sure_refer_info(sure_refer_text_data)

In [7]:
sure_refer_text_list

[['ชัวร์ก่อนแชร์: ตาลีบันห้ามฉีดวัคซีนโควิด 19 ในอัฟกานิสถาน จริงหรือ?',
  'https://tna.mcot.net/latest-news-825069'],
 ['ชัวร์ก่อนแชร์: WHO ประกาศให้โควิด 19 เป็นโรคประจำถิ่น จริงหรือ?',
  'https://tna.mcot.net/latest-news-824200'],
 ['ชัวร์ก่อนแชร์: WHO ไม่ยอมรับภูมิคุ้มกันธรรมชาติ จริงหรือ?',
  'https://tna.mcot.net/latest-news-823234'],
 ['ชัวร์ก่อนแชร์: สหรัฐใช้ยา Ivermectin รักษาผู้อพยพที่ติดเชื้อโควิด จริงหรือ?',
  'https://tna.mcot.net/sureandshare-822486'],
 ['ชัวร์ก่อนแชร์: ญี่ปุ่นยกเลิกวัคซีน Moderna ใช้ Ivermectin รักษาโควิด จริงหรือ?',
  'https://tna.mcot.net/latest-news-821794'],
 ['ชัวร์ก่อนแชร์: Oxford รับรองการรักษาโควิด 19 ด้วย Ivermectin จริงหรือ?',
  'https://tna.mcot.net/latest-news-821239'],
 ['ชัวร์ก่อนแชร์: หัวหน้านักวิทย์ WHO โดนดำเนินคดีกรณีด้อยค่า Ivermectin จริงหรือ?',
  'https://tna.mcot.net/latest-news-820342'],
 ['ชัวร์ก่อนแชร์: สวิตเซอร์แลนด์จำหน่ายยากำจัดวัคซีนโควิด 19 จากร่างกาย จริงหรือ?',
  'https://tna.mcot.net/latest-news-819351'],
 ['ชัวร์ก่อนแชร์

In [8]:
len(sure_refer_text_list)

2068

## ------------------------------------------------------------------------------
## สร้าง dataframe ของ cofact สำหรับอ้างถึงตอนแนบลิงค์ url แสดงผล

In [9]:
sure_refer_text = []
sure_refer_url = []
for i in range(len(sure_refer_text_list)):
    sure_refer_text.append(sure_refer_text_list[i][0]) #list ของส่วนเนื้อหาเท่านั้น
    sure_refer_url.append(sure_refer_text_list[i][1]) #list ของ url เท่านั้น


In [10]:
#ทำ list ให้เป็น dataframe
sure_original_text_and_headline_news_df = pd.DataFrame(list(zip(sure_refer_text, sure_refer_url)), columns=["หัวข้อข่าวอ้างอิงของ sure fake-news", "URL ของ sure fake-news"])

sure_original_text_and_headline_news_df

,หัวข้อข่าวอ้างอิงของ sure fake-news,URL ของ sure fake-news
0,ชัวร์ก่อนแชร์: ตาลีบันห้ามฉีดวัคซีนโควิด 19 ใน...,https://tna.mcot.net/latest-news-825069
1,ชัวร์ก่อนแชร์: WHO ประกาศให้โควิด 19 เป็นโรคปร...,https://tna.mcot.net/latest-news-824200
2,ชัวร์ก่อนแชร์: WHO ไม่ยอมรับภูมิคุ้มกันธรรมชาต...,https://tna.mcot.net/latest-news-823234
3,ชัวร์ก่อนแชร์: สหรัฐใช้ยา Ivermectin รักษาผู้อ...,https://tna.mcot.net/sureandshare-822486
4,ชัวร์ก่อนแชร์: ญี่ปุ่นยกเลิกวัคซีน Moderna ใช้...,https://tna.mcot.net/latest-news-821794
...,...,...
2063,ชัวร์ก่อนแชร์ : ระวัง 10 ขนมขบเคี้ยวอันตราย! จ...,https://tna.mcot.net/tna-93789
2064,ชัวร์ก่อนแชร์ : เปิดแอร์ใส่หน้าเด็ก ทำให้เป็นอ...,https://tna.mcot.net/tna-93790
2065,ชัวร์ก่อนแชร์ : มะเร็งต้องเลิกบำรุงจริงหรือ?,https://tna.mcot.net/tna-93791
2066,ชัวร์ก่อนแชร์ : ยาแอสไพรินช่วยซักผ้าขาวสะอาด จ...,https://tna.mcot.net/tna-100770


In [11]:
sure_headline_tokens_list = [split_word(txt) for txt in sure_refer_text] #นำ list ของเนื้อหามาตัดคำ

In [12]:
sure_tokens_list_j = [','.join(tkn) for tkn in sure_headline_tokens_list]

----------------------------------------------------------------------------------
# สร้าง vocabulary set

In [13]:
print(sure_tokens_list_j[0].split(",")[0:2])


['ชัวร์', 'แชร์']


In [14]:
## Create Vocabulary
tokens_list = []

for words in sure_tokens_list_j:
    # print(words)
    temp_list = words.split(",")
    # print(temp_list)
    for i in temp_list:
        tokens_list.append(i)

vocabulary = set(tokens_list)
vocabulary = list(vocabulary)

In [15]:
len(vocabulary)

2561

--------------------------------------------------------------------------------------

In [16]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
original_c_feat = tvec.fit_transform(sure_tokens_list_j)

### Save Trained TFID Model

In [17]:
### Save model
root_path = os.getcwd()
path = os.path.join(root_path, 'sure-tfid.pkl')
print(path)
with open(path,'wb') as handle:
    pickle.dump(original_c_feat, handle)

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\sure-tfid.pkl


In [18]:
### load model
root_path = os.getcwd()
path = os.path.join(root_path, 'sure-tfid.pkl')
print(path)
t = pickle.load(open(path,'rb'))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\sure-tfid.pkl


### Save Vocabulary

In [19]:
### Save Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_sure.txt')
print(path)
with open(path, "w", encoding="utf-8") as file:
    file.write(str(vocabulary))

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_sure.txt


In [20]:
### load Vacabulary
root_path = os.getcwd()
path = os.path.join(root_path, 'vocabulary_sure.txt')
print(path)
with open(path, "r", encoding="utf-8") as file:
    data2 = eval(file.readline())

c:\Users\Administrator\Documents\SeniorProject\Senior-project\Pre-process\vocabulary_sure.txt


In [21]:
data2

['การทดลอง',
 'Luciferin',
 'ใช้กับ',
 'กข',
 'เส้นประสาท',
 'เช็ก',
 'ประจำเดือน',
 'ซ้ำ',
 'Facts',
 'หน้าแก่',
 'แก๊ง',
 'ผม',
 'นิค',
 'ขโมย',
 'กินเจ',
 'เลิก',
 'ยาคุม',
 'สาร',
 'ครรภ์',
 'สาเหตุ',
 'ไลฟ์',
 'ฝาก',
 'พญาสัตบรรณ',
 'อยู่รอด',
 'ไหว',
 'จำหน่าย',
 'ตะไบ',
 'มิเตอร์',
 'กระเบน',
 'เหรียญ',
 'แนะ',
 'จืด',
 'หมอ',
 'จร',
 'แยก',
 'เนื้องอก',
 'แผล',
 'ถ่าน',
 'ป้าย',
 'พายุ',
 'ก่อ',
 'ถ่ายทอด',
 'รถ',
 'ให้อาหาร',
 'แบบทดสอบ',
 'ต้นแขน',
 'ผิดปกติ',
 'เคาะ',
 'อา',
 'น้ำชา',
 'อาการป่วย',
 'เครา',
 'เหงา',
 'ลุกลาม',
 'ป่วยหนัก',
 'คันเร่ง',
 'เบตาดีน',
 'โรคติดต่อ',
 'ชะลอ',
 'สัตว์',
 'ยาคุมกำเนิด',
 'เตาแก๊ส',
 'รอง',
 'รี่',
 'เครื่องซักผ้า',
 'เตือน',
 'เส้น',
 'คดี',
 'ด้อยค่า',
 'เป็นตัว',
 'ฉลาด',
 'แนวปฏิบัติ',
 'ลม',
 'Start',
 'เรื่อง',
 'ลูกเล่น',
 'ไหล่',
 'ทำร้าย',
 'ดก',
 'เคล็ด',
 'ถิ่น',
 'ฝัง',
 'ภาพซ้อน',
 'เจียระไน',
 'พิการ',
 'สั่งห้าม',
 'คอนโด',
 'เป็นสาเหตุ',
 'ตรงไหน',
 'ผ่าตัด',
 'แท้งลูก',
 'คู่',
 'เทป',
 'ขุ่น',
 'ร้าย',
 'เทียม',
 'ข้

In [22]:
len(data2)

2561

## Create vector for Query/search keywords

In [23]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x = tvec.transform(tokens)
    
    x = x.A[0]
    return x

### Calculate Cosine similarity of trained Tfidf to input query


In [24]:
query_vector = []
def cosine_similarity_T(k, query):
    tokens = split_word(str(query))
    q_df = pd.DataFrame(columns=['q_clean'])
    # print(tokens)
    q_df.loc[0,'q_clean'] =str(tokens)

    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in original_c_feat.A:
        d_cosines.append(1 - spatial.distance.cosine(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    d_cosines.sort()

    a = pd.DataFrame()
    for i in out:
        a.loc[i,'index'] = str(i)
        a.loc[i,'Subject'] = sure_original_text_and_headline_news_df['หัวข้อข่าวอ้างอิงของ sure fake-news'][i]

    d_cosines.sort(reverse=True)
    
    for j in range(k):
        a.loc[out[j],'Score'] = d_cosines[j]
        
    return a

In [25]:
%time sure_result = cosine_similarity_T(10,'มีกาแฟผสมยาอี  กำลังระบาดในกลุ่มวัยรุ่นมีกาแฟผสมยาอี กำลังระบาดในกลุ่มวัยรุ่น')

Wall time: 116 ms


In [26]:
sure_result

,index,Subject,Score
1282,1282,ชัวร์ก่อนแชร์ : คลิปเตือนกาแฟผสมดินประสิว จริง...,0.410252
1872,1872,ชัวร์ก่อนแชร์ : 20 ประโยชน์ของกาแฟ จริงหรือ?,0.360431
1655,1655,ชัวร์ก่อนแชร์ : สูตรกาแฟผสมนมเปรี้ยว ไล่แมลงได...,0.310751
1560,1560,ชัวร์ก่อนแชร์ : ดื่มกาแฟอย่างไรไม่ให้เสียให้สุ...,0.309183
996,996,ชัวร์ก่อนแชร์ : 5 โรคห้ามดื่มกาแฟ จริงหรือ?,0.296173
1356,1356,ชัวร์ก่อนแชร์ : ดื่มน้ำร้อนและกาแฟ รักษาหวัด ไ...,0.267722
89,89,ชัวร์ก่อนแชร์ LIVE : 28 ก.ค. 64 โควิดระบาด กิน...,0.254570
461,461,ชัวร์ก่อนแชร์ LIVE : 28 ก.ค. 64 โควิดระบาด กิน...,0.254570
1586,1586,ชัวร์ก่อนแชร์ : งูสวัดระบาดจริงหรือ ?,0.248179
1938,1938,ชัวร์ก่อนแชร์ : ชามีกาเฟอีนมากกว่ากาแฟจริงหรือ?,0.246952


In [27]:
sure_result_with_url = pd.DataFrame()
for i in range(len(sure_result)):
    sure_result_with_url.loc[i,'index'] = sure_result.iloc[i]["index"]
    sure_result_with_url.loc[i,'text'] = sure_result.iloc[i]["Subject"]
    sure_result_with_url.loc[i,'url'] = sure_original_text_and_headline_news_df["URL ของ sure fake-news"][int(sure_result.iloc[i]["index"])]

In [28]:
sure_result_with_url

,index,text,url
0,1282,ชัวร์ก่อนแชร์ : คลิปเตือนกาแฟผสมดินประสิว จริง...,https://tna.mcot.net/tna-278578
1,1872,ชัวร์ก่อนแชร์ : 20 ประโยชน์ของกาแฟ จริงหรือ?,https://tna.mcot.net/tna-75433
2,1655,ชัวร์ก่อนแชร์ : สูตรกาแฟผสมนมเปรี้ยว ไล่แมลงได...,https://tna.mcot.net/tna-167238
3,1560,ชัวร์ก่อนแชร์ : ดื่มกาแฟอย่างไรไม่ให้เสียให้สุ...,https://tna.mcot.net/tna-197777
4,996,ชัวร์ก่อนแชร์ : 5 โรคห้ามดื่มกาแฟ จริงหรือ?,https://tna.mcot.net/tna-370528
5,1356,ชัวร์ก่อนแชร์ : ดื่มน้ำร้อนและกาแฟ รักษาหวัด ไ...,https://tna.mcot.net/tna-256597
6,89,ชัวร์ก่อนแชร์ LIVE : 28 ก.ค. 64 โควิดระบาด กิน...,https://tna.mcot.net/latest-news-750999
7,461,ชัวร์ก่อนแชร์ LIVE : 28 ก.ค. 64 โควิดระบาด กิน...,https://tna.mcot.net/latest-news-750999
8,1586,ชัวร์ก่อนแชร์ : งูสวัดระบาดจริงหรือ ?,https://tna.mcot.net/tna-190644
9,1938,ชัวร์ก่อนแชร์ : ชามีกาเฟอีนมากกว่ากาแฟจริงหรือ?,https://tna.mcot.net/tna-9776


---------------------------------------